In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import jdatetime
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    mean_squared_error, mean_absolute_error, 
    r2_score, mean_absolute_percentage_error
)
from sklearn.preprocessing import MinMaxScaler
from torch.utils.data import TensorDataset, DataLoader
import plotly.express as px
import plotly.graph_objects as go
from tqdm import tqdm
import torch
from torch import nn
import torch.optim as optim

In [2]:
df = pd.read_csv('../data/processed/economics.csv')
df.head()

,Date,b1,b2,b3,b4,b5,b7,b8,b9,b10,b14,b17,b18,B-23,B-24
0,1360-01-01,2806.0,NaN,918.0,4380.0,25.1,17.0,19.0,77.7,270.0,1078.5,NaN,NaN,NaN,NaN
1,1360-04-01,2437.0,NaN,834.0,4552.0,20.8,18.0,19.0,81.3,270.0,1191.1,NaN,NaN,NaN,NaN
2,1360-07-01,2600.0,NaN,720.0,4653.8,23.3,18.0,20.0,79.6,270.0,1246.2,NaN,NaN,NaN,NaN
3,1360-10-01,2355.0,NaN,750.0,5236.1,18.4,20.0,19.0,80.8,270.0,1408.1,NaN,NaN,NaN,NaN
4,1361-01-01,2332.0,NaN,850.0,5250.2,44.8,20.0,22.0,82.4,350.0,1324.9,0.5,0.5,0.233333,NaN


In [3]:
df['Date'] = df['Date'].apply(lambda x: jdatetime.datetime.strptime(x, '%Y-%m-%d'))
df['Date'] = df['Date'].apply(lambda x: x.togregorian())
df.head()

,Date,b1,b2,b3,b4,b5,b7,b8,b9,b10,b14,b17,b18,B-23,B-24
0,1981-03-21,2806.0,NaN,918.0,4380.0,25.1,17.0,19.0,77.7,270.0,1078.5,NaN,NaN,NaN,NaN
1,1981-06-22,2437.0,NaN,834.0,4552.0,20.8,18.0,19.0,81.3,270.0,1191.1,NaN,NaN,NaN,NaN
2,1981-09-23,2600.0,NaN,720.0,4653.8,23.3,18.0,20.0,79.6,270.0,1246.2,NaN,NaN,NaN,NaN
3,1981-12-22,2355.0,NaN,750.0,5236.1,18.4,20.0,19.0,80.8,270.0,1408.1,NaN,NaN,NaN,NaN
4,1982-03-21,2332.0,NaN,850.0,5250.2,44.8,20.0,22.0,82.4,350.0,1324.9,0.5,0.5,0.233333,NaN


In [4]:
# Date to Unix timestamp
df['Date'] = pd.to_datetime(df['Date']).astype(int) / 10**9
df.head()

,Date,b1,b2,b3,b4,b5,b7,b8,b9,b10,b14,b17,b18,B-23,B-24
0,353980800.0,2806.0,NaN,918.0,4380.0,25.1,17.0,19.0,77.7,270.0,1078.5,NaN,NaN,NaN,NaN
1,362016000.0,2437.0,NaN,834.0,4552.0,20.8,18.0,19.0,81.3,270.0,1191.1,NaN,NaN,NaN,NaN
2,370051200.0,2600.0,NaN,720.0,4653.8,23.3,18.0,20.0,79.6,270.0,1246.2,NaN,NaN,NaN,NaN
3,377827200.0,2355.0,NaN,750.0,5236.1,18.4,20.0,19.0,80.8,270.0,1408.1,NaN,NaN,NaN,NaN
4,385516800.0,2332.0,NaN,850.0,5250.2,44.8,20.0,22.0,82.4,350.0,1324.9,0.5,0.5,0.233333,NaN


In [5]:
df.dtypes

Date    float64
b1      float64
b2      float64
b3      float64
b4      float64
b5      float64
b7      float64
b8      float64
b9      float64
b10     float64
b14     float64
b17     float64
b18     float64
B-23    float64
B-24    float64
dtype: object

In [6]:
df['b8'] = df['b8'].fillna(method='ffill')
df.isna().sum() / len(df)

Date    0.000000
b1      0.046512
b2      0.255814
b3      0.046512
b4      0.046512
b5      0.046512
b7      0.046512
b8      0.000000
b9      0.046512
b10     0.046512
b14     0.046512
b17     0.069767
b18     0.069767
B-23    0.069767
B-24    0.279070
dtype: float64

In [7]:
# normalize the data
scaler = MinMaxScaler(feature_range=(-1, 1))

df_cols = df.columns

df = scaler.fit_transform(df)

df = pd.DataFrame(df, columns=df_cols)
df.head()

,Date,b1,b2,b3,b4,b5,b7,b8,b9,b10,b14,b17,b18,B-23,B-24
0,-1.000000,-0.664623,NaN,-0.965079,-1.000000,-0.999941,-1.000000,-1.000000,-0.999380,-1.000000,-1.000000,NaN,NaN,NaN,NaN
1,-0.988089,-0.745403,NaN,-0.979894,-0.999993,-0.999978,-0.999971,-1.000000,-0.999211,-1.000000,-0.999739,NaN,NaN,NaN,NaN
2,-0.976178,-0.709720,NaN,-1.000000,-0.999989,-0.999956,-0.999971,-0.999977,-0.999291,-1.000000,-0.999611,NaN,NaN,NaN,NaN
3,-0.964652,-0.763354,NaN,-0.994709,-0.999965,-0.999998,-0.999912,-1.000000,-0.999234,-1.000000,-0.999236,NaN,NaN,NaN,NaN
4,-0.953253,-0.768389,NaN,-0.977072,-0.999964,-0.999772,-0.999912,-0.999932,-0.999159,-0.999422,-0.999429,-1.0,-1.0,-0.999861,NaN


In [9]:
# split data to train and test
# x_train, x_test, y_train, y_test = train_test_split(
#     df[[c for c in df if c not in ['y']]], df['y'], test_size=0.2, random_state=42
# )
x_train, x_test, y_train, y_test = train_test_split(
    df['Date'], df['b8'], test_size=0.15, random_state=42
)
x_train, x_val, y_train, y_val = train_test_split(
    x_train, y_train, test_size=0.15, random_state=42
)

print(x_train.shape, y_train.shape, x_val.shape, y_val.shape, x_test.shape, y_test.shape)

(124,) (124,) (22,) (22,) (26,) (26,)


In [10]:
x_train_normalized = scaler.fit_transform(x_train.values.reshape(-1, 1))
x_val_normalized = scaler.transform(x_val.values.reshape(-1, 1))
x_test_normalized = scaler.transform(x_test.values.reshape(-1, 1))

# Reshape data from [samples, features] to [samples, timesteps, features] 
x_train_final = x_train_normalized.reshape(-1, 1, 1)
x_val_final = x_val_normalized.reshape(-1, 1, 1)
x_test_final = x_test_normalized.reshape(-1, 1, 1)

x_train_tensor = torch.from_numpy(x_train_final).float()
y_train_tensor = torch.from_numpy(y_train.values).float()
x_val_tensor = torch.from_numpy(x_val_final).float()
y_val_tensor = torch.from_numpy(y_val.values).float()
x_test_tensor = torch.from_numpy(x_test_final).float()
y_test_tensor = torch.from_numpy(y_test.values).float()

train_data = TensorDataset(x_train_tensor, y_train_tensor)
train_loader = DataLoader(train_data, batch_size=32)
val_data = TensorDataset(x_val_tensor, y_val_tensor)
val_loader = DataLoader(val_data, batch_size=32)
test_data = TensorDataset(x_test_tensor, y_test_tensor)
test_loader = DataLoader(test_data, batch_size=32)

In [11]:
class Model(nn.Module):
    def __init__(self, feature_size, num_layers, dropout=0.1):
        super(Model, self).__init__()
        self.lstm = nn.LSTM(feature_size, feature_size, num_layers=num_layers, batch_first=True)
        transformer_layer = nn.TransformerEncoderLayer(d_model=feature_size, nhead=1, dropout=dropout)
        self.transformer = nn.TransformerEncoder(transformer_layer, num_layers=num_layers)
        self.fc = nn.Linear(feature_size, 1)

    def forward(self, x):
        x, _ = self.lstm(x)
        x = self.transformer(x)
        x = self.fc(x)
        return x

In [15]:
model = Model(feature_size=1, num_layers=10, dropout=0.1)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.MSELoss()

num_epochs = 256
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

train_losses = []
val_losses = []

# Training loop
for epoch in range(num_epochs):
    model.train() 
    train_loss = 0
    for batch in train_loader:
        inputs, labels = batch
        inputs = inputs.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        train_loss += loss.item()

        # Backward pass and optimizations
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    avg_train_loss = train_loss / len(train_loader)
    train_losses.append(avg_train_loss)

    # Validation loop
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for batch in val_loader:
            inputs, labels = batch
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

    avg_val_loss = val_loss / len(val_loader)
    val_losses.append(avg_val_loss)

    print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}')

/home/smbh/.venvs/house-price-prediction/lib/python3.11/site-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([32])) that is different to the input size (torch.Size([32, 1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/home/smbh/.venvs/house-price-prediction/lib/python3.11/site-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([28])) that is different to the input size (torch.Size([28, 1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/home/smbh/.venvs/house-price-prediction/lib/python3.11/site-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([22])) that is different to the input size (torch.Size([22, 1, 1])). This will likely lead to incor

Epoch 1/256, Train Loss: 0.3837, Val Loss: 0.5236
Epoch 2/256, Train Loss: 0.3780, Val Loss: 0.5197
Epoch 3/256, Train Loss: 0.3724, Val Loss: 0.5160
Epoch 4/256, Train Loss: 0.3668, Val Loss: 0.5123
Epoch 5/256, Train Loss: 0.3613, Val Loss: 0.5087
Epoch 6/256, Train Loss: 0.3560, Val Loss: 0.5052
Epoch 7/256, Train Loss: 0.3507, Val Loss: 0.5018
Epoch 8/256, Train Loss: 0.3455, Val Loss: 0.4985
Epoch 9/256, Train Loss: 0.3404, Val Loss: 0.4953
Epoch 10/256, Train Loss: 0.3354, Val Loss: 0.4922
Epoch 11/256, Train Loss: 0.3306, Val Loss: 0.4892
Epoch 12/256, Train Loss: 0.3258, Val Loss: 0.4863
Epoch 13/256, Train Loss: 0.3211, Val Loss: 0.4835
Epoch 14/256, Train Loss: 0.3166, Val Loss: 0.4808
Epoch 15/256, Train Loss: 0.3121, Val Loss: 0.4783
Epoch 16/256, Train Loss: 0.3078, Val Loss: 0.4758
Epoch 17/256, Train Loss: 0.3036, Val Loss: 0.4734
Epoch 18/256, Train Loss: 0.2995, Val Loss: 0.4711
Epoch 19/256, Train Loss: 0.2955, Val Loss: 0.4690
Epoch 20/256, Train Loss: 0.2916, Val Lo

In [18]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=list(range(num_epochs)), y=train_losses,
                    mode='lines',
                    name='training loss'))
fig.add_trace(go.Scatter(x=list(range(num_epochs)), y=val_losses,
                    mode='lines',
                    name='validation loss'))

fig.update_layout(title='Training and Validation Losses',
                   xaxis_title='Epochs',
                   yaxis_title='Loss')
fig.show()


In [19]:
# Evaluation loop
model.eval()
total_loss = 0
predictions = []
actuals = []
with torch.no_grad():
    for batch in val_loader:
        inputs, labels = batch
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        total_loss += loss.item()

        predictions.extend(outputs.view(-1).cpu().numpy())
        actuals.extend(labels.view(-1).cpu().numpy())

avg_val_loss = total_loss / len(val_loader)
print(f'Validation Loss: {avg_val_loss}')

# Convert lists to numpy arrays
predictions = np.array(predictions)
actuals = np.array(actuals)

# Compute metrics
mse = mean_squared_error(actuals, predictions)
mae = mean_absolute_error(actuals, predictions)
r2 = r2_score(actuals, predictions)

print(f'Mean Squared Error: {mse}')
print(f'Mean Absolute Error: {mae}')
print(f'R^2 Score: {r2}')

Validation Loss: 0.4609185457229614
Mean Squared Error: 0.46091702580451965
Mean Absolute Error: 0.367571622133255
R^2 Score: -0.038682118194598214


/home/smbh/.venvs/house-price-prediction/lib/python3.11/site-packages/torch/nn/modules/loss.py:536: UserWarning:

Using a target size (torch.Size([22])) that is different to the input size (torch.Size([22, 1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.

